In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import l2beat_utils as ltwo
sys.path.pop()

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# # Get token classification
# # https://dune.com/queries/3852319
token_df = d.get_dune_data(query_id = 3852319,
    name = "tokens_by_type_base_opm",
    performance = 'large'#,
    # num_hours_to_rerun=0
)

token_df.sample(5)

2024-06-21 19:50:08.377 | INFO     | duneapi_utils:get_dune_data:58 - Results available at https://dune.com/queries/3852319
2024-06-21 19:50:08,911 INFO dune_client.base_client executing 3852319 on large cluster
2024-06-21 19:50:09,847 INFO dune_client.base_client waiting for query execution 01J0YK9XNJ4H5350317G2JTZDD to complete: ExecutionState.PENDING (queue position: 1)
2024-06-21 19:50:15,319 INFO dune_client.base_client waiting for query execution 01J0YK9XNJ4H5350317G2JTZDD to complete: ExecutionState.EXECUTING
2024-06-21 19:50:20,886 INFO dune_client.base_client waiting for query execution 01J0YK9XNJ4H5350317G2JTZDD to complete: ExecutionState.EXECUTING
2024-06-21 19:50:26,337 INFO dune_client.base_client waiting for query execution 01J0YK9XNJ4H5350317G2JTZDD to complete: ExecutionState.EXECUTING
2024-06-21 19:50:31,804 INFO dune_client.base_client waiting for query execution 01J0YK9XNJ4H5350317G2JTZDD to complete: ExecutionState.EXECUTING
2024-06-21 19:50:37,229 INFO dune_client

,blockchain,chain_id,factory_address,l1_token,symbol,token_address,token_classification,last_updated
161833,base,8453,None,None,MADPEPE,0x1368ac8a64709b0f5238b52889764ada855d80f0,Not Proxy or Factory,2024-06-21 23:49:59.603507
161462,base,8453,None,None,SBF,0xa3b393e5f803d0add3f95421ae5bfe0acaa447f0,Not Proxy or Factory,2024-06-21 23:49:59.603507
233681,base,8453,None,None,DGZQ,0x0c4c1e9268c6635beb48f78c4cbbf5488154da1d,Not Proxy or Factory,2024-06-21 23:49:59.603507
166780,base,8453,None,None,BKRW,0x24917a5973620972ad1917e08d13e46f25098de7,Not Proxy or Factory,2024-06-21 23:49:59.603507
307377,base,8453,None,None,GOJAK,0xf9fea26616ca0f3b46f208ce4495d896a27ef3cb,Not Proxy or Factory,2024-06-21 23:49:59.603507


In [3]:
# Get Lastest L2B aoc
l2b_df = ltwo.get_daily_aoc_by_token()
l2b_df_spc = l2b_df[l2b_df['project'].isin(['optimism','base'])]
l2b_df_spc = l2b_df_spc.rename(columns={'project':'blockchain'})

l2b_df_spc = l2b_df_spc[['dt','blockchain','token_type','asset_id','address','usd_value']]
l2b_df_spc['address'] = l2b_df_spc['address'].str.lower()

l2b_df_spc.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value
759,2024-06-21,optimism,canonical,sdai-savings-dai,0x83f20f44975d03b1b09e64809b757c47f942beea,162516.29
751,2024-06-21,optimism,canonical,zrx-0x-protocol-token,0xe41d2489571d322189246dafa5ebde1f4699f498,150808.31
781,2024-06-21,optimism,canonical,hair-hairdao-token,0x9ce115f0341ae5dabc8b477b74e83db2018a6f42,2683698.65
785,2024-06-21,optimism,canonical,cyber-cyberconnect,0x14778860e937f509e651192a90589de711fb88a9,5.34
242,2024-06-21,base,canonical,comp-compound,0xc00e94cb662c3520282e6f5717214004a7f26888,576669.52


In [4]:
cols = ['blockchain','token_address','l1_token','token_classification','symbol']
token_if_l1_df = token_df[cols].rename(columns={'l1_token':'address'})
token_if_l2_df = token_df[cols].rename(columns={'token_address':'address'})

In [5]:
df_cols = ['blockchain','address','token_classification']
token_if_l1_df = token_if_l1_df[~token_if_l1_df['address'].isna()][df_cols].drop_duplicates()
token_if_l2_df = token_if_l2_df[~token_if_l2_df['address'].isna()][df_cols].drop_duplicates()

token_if_l2_df.sample(5)

,blockchain,address,token_classification
307902,base,0x79cbc289fe7cc4db4a72a57551f5c95cb4b52c84,Not Proxy or Factory
245789,base,0xc2498a59e49aff93c2954b58704309ab1533f12e,Not Proxy or Factory
225551,base,0x9abe6da694dae8b0b8ae2a53012219f47ca11e3b,Not Proxy or Factory
73792,base,0x8c60fe6df2247d552970c9ed256e3c98a182e024,Not Proxy or Factory
191377,base,0x9d55ffdca5e21f4bb2b29dca12577535672f690a,Not Proxy or Factory


In [6]:
#L2 Mapping preferred
unified_df = l2b_df_spc.merge(token_if_l2_df
                              , on=['blockchain','address'], how='left')
#L1 Mapping fallback
unified_df = unified_df.merge(token_if_l1_df
                              , on=['blockchain','address'], how='left')


In [7]:
unified_df['token_classification'] = unified_df['token_classification_x'].fillna(unified_df['token_classification_y']).fillna('Unknown')

unified_df = unified_df.drop(columns=['token_classification_x', 'token_classification_y'])

In [8]:
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Native' if row['asset_id'] == 'eth-ether' else row['token_classification'], axis=1
)
# Override due to bad Base events
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Proxy' if row['asset_id'] == 'base:usdc-usd-coin' else row['token_classification'], axis=1
)
# Override OP predeploy
unified_df['token_classification'] = unified_df.apply(
    lambda row: 'Predeploy' if row['asset_id'] == 'op-optimism' else row['token_classification'], axis=1
)

In [ ]:
# Define the priority order for token_classification
priority = {'Proxy': 1, 'Not Proxy or Factory': 2, 'Immutable Factory Created': 3}

# Add a priority column to the DataFrame based on the token_classification
unified_df['priority'] = unified_df['token_classification'].map(priority)

# Sort the DataFrame by the priority column
unified_df = unified_df.sort_values(by='priority')

# Drop duplicates based on the unique columns, keeping the first occurrence (which has the highest priority)
unique_columns = ['dt', 'blockchain', 'token_type', 'asset_id', 'address']
unified_df = unified_df.drop_duplicates(subset=unique_columns, keep='first')

# Drop the priority column as it's no longer needed
unified_df = unified_df.drop(columns=['priority'])

In [9]:
unified_df.to_csv('csv_outputs/aoc_by_chain_token_classification.csv')

In [10]:
unified_df[unified_df['address']=='0x833589fcd6edb6e08f4c7c32d4f71b54bda02913']#.sample(5)

,dt,blockchain,token_type,asset_id,address,usd_value,token_classification
44,2024-06-21,base,native,base:usdc-usd-coin,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,3.055468e+09,Proxy
